In [ ]:
from functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os

In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)
alpha = 0.3
r2 = 0.925

L_limit = 4
n_precision = 10000
X_precision = np.linspace(-L_limit, L_limit, n_precision)
N_theta = 20
N_bins = 30

In [ ]:
N_mu = (np.log(707))**2
print(N_mu)

In [ ]:
grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)

In [ ]:
trial_bin = 1

xb = np.array(grouped_data_dict[trial_bin])
W_Gaussian, bw_Gaussian = pdf_kernel_optimal_bandwidth_histogram(X_precision, xb, kernel_method = 'gaussian', bandwidth_method = 'silverman')
W_Epanechnikov, bw_Epanechnikov = pdf_kernel_optimal_bandwidth_histogram(X_precision, xb, kernel_method = 'epanechnikov', bandwidth_method = 'silverman')

hist_counts, bin_edges = np.histogram(grouped_data_dict[trial_bin], bins=N_bins, density=True)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
plt.bar(bin_centers, hist_counts, width=(bin_edges[1] - bin_edges[0]), alpha=0.6, label="Experimental Data", color="gray", edgecolor="black")
plt.plot(X_precision, P_x_theta(X_precision, grouped_theta_list[trial_bin], alpha, r2), label="Theoretical PDF", color="red", linewidth=2)
plt.plot(X_precision, W_Gaussian, color = 'blue', label="Gaussian Kernel")
plt.plot(X_precision, W_Epanechnikov, color = 'green', label="Epanechnikov Kernel")

plt.legend()
plt.xlabel(r"$x$")
plt.ylabel(r"$p(x|\theta)$")

plt.title(f"Tomogram, $θ$ = {grouped_theta_list[trial_bin]:.2f}")
plt.xlim(-L_limit, L_limit)

In [ ]:
error = []
for trial_bin in range(N_theta):
    xb = np.array(grouped_data_dict[trial_bin])
    W_Gaussian, bw_Gaussian = pdf_kernel_optimal_bandwidth_histogram(X_precision, xb, kernel_method = 'gaussian', bandwidth_method = 'silverman')
    P_theoretical = P_x_theta(X_precision, grouped_theta_list[trial_bin], alpha, r2)
    error_step = simpson(y = (P_theoretical - W_Gaussian)**2, x = X_precision)
    error.append(error_step)

np.mean(error)